<a href="https://colab.research.google.com/github/manujsinghwal/23andMe-data-analysis/blob/main/23andMe_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 23andMe Data Analysis

In [2]:
# Mounting Google drive as dataset stored in a drive directory
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Data Exploration
Before diving into the questions, let's apply some data exploration techniques to have a better understanding of what we are dealing with in the dataset. We'll start with importing a few useful libraries. We will use pandas to create and analyse the dataset, and matplotlib.pyplot to do plot graphs to visualize our answers. Glob will be used just to get all filenames under the /data directory.

In [1]:
# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import glob

Let's create our dataset. First, we need to get all filenames under the **23andMe** directory in a list so we can concatenate them in one dataset using the pandas concat function which can accept a list as input. This method prevents us to use a for loop to iterate over files, read them separately and append to a dataset.

In [12]:
# Getting all filenames under the data directory
l = [pd.read_csv(filename) for filename in glob.glob("gdrive/My Drive/Colab Notebooks/Dataset/23andMe/*.csv")]

# Checking the list size to understand how many we read using above statement
len(l)

50

We have 50 files in our dataset, hence it's matching with the number of total files over there. Now, let's concatnate the data from these files and get everthing in a single dataframe.

In [7]:
# Creating the dataset using all files under the data directory
df = pd.concat(l, axis = 0)
df.head()

,sale_time,purchaser_gender
0,2012-11-05 01:03:52,female
1,2012-11-05 01:17:49,female
2,2012-11-05 01:37:42,female
3,2012-11-05 01:43:03,female
4,2012-11-05 01:54:20,female


In [8]:
df.columns

Index(['sale_time', 'purchaser_gender'], dtype='object')

In [9]:
df.shape

(204329, 2)

In [10]:
# Assigning a constant figure size and use it in plotting to make plots larger
FIG_SIZE = (8,6)